In [ ]:
from transformers import pipeline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import tqdm

In [ ]:
device = 'mps'

## 2. Using Transformer

### 2.2 Behind the pipeline

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)
seq_class = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
model.to(device);
seq_class.to(device);

In [ ]:
total = 0
for n, p in model.named_parameters():
    total += p.numel()
total

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt").to(device)

In [ ]:
inputs['input_ids']

In [ ]:
inputs['input_ids'].shape

In [ ]:
inputs['attention_mask']

In [ ]:
inputs['attention_mask'].shape

In [ ]:
outputs = model(**inputs)

In [ ]:
outputs.last_hidden_state.shape

In [ ]:
outputs = seq_class(**inputs)

In [ ]:
outputs.logits

In [ ]:
scipy.special.softmax(outputs.logits.detach().cpu(), axis=1)

In [ ]:
seq_class.config.id2label

### 2.3 Models

In [ ]:
from transformers import BertConfig, BertTokenizer, BertModel

In [ ]:
# Building the config
config = BertConfig()

# Building the model from the config
model = BertModel(config)

In [ ]:
config

In [ ]:
name_params = []
for n, p in model.named_parameters():
    name_params.append([n, p.numel()])

In [ ]:
df = pd.DataFrame(name_params, columns=['names', 'parameters'])

In [ ]:
df['parameters'].sum()

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
bert_model = BertModel.from_pretrained('bert-base-cased')

In [ ]:
name_params = []
for n, p in bert_model.named_parameters():
    name_params.append([n, p.numel()])

In [ ]:
df = pd.DataFrame(name_params, columns=['names', 'parameters'])

In [ ]:
df['parameters'].sum()

In [ ]:
bert_model.to(device);

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs_bert = bert_tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt").to(device)

In [ ]:
inputs['input_ids'].shape

In [ ]:
inputs_bert['input_ids'].shape

In [ ]:
outputs = bert_model(**inputs_bert)

In [ ]:
outputs.last_hidden_state.shape

In [ ]:
#bert.save_pretrained('test')

### 2.4 Tokenizers

In [ ]:
from transformers import BertTokenizer

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
sentence = 'Using a Transformer network is simple'

In [ ]:
inputs = bert_tokenizer(sentence)

In [ ]:
inputs

In [ ]:
#bert_tokenizer.save_pretrained('test')

In [ ]:
vocab = bert_tokenizer.vocab

In [ ]:
vocab_sort = np.sort(list(vocab.keys()))

In [ ]:
vocab_sort.shape

In [ ]:
vocab_lower = np.unique(np.char.lower(vocab_sort[7245:-918]))

In [ ]:
vocab_lower.shape

In [ ]:
sentence = 'Using a Transformer network is simple'

In [ ]:
tokens = bert_tokenizer.tokenize(sentence)

In [ ]:
tokens

In [ ]:
ids = bert_tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
ids

In [ ]:
decoded_string = bert_tokenizer.decode(ids)

In [ ]:
decoded_string

### 2.5 Handling multiple sequences

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
model.to(device);

In [ ]:
sequence = "I've been waiting for a HuggingFace course my whole life."

In [ ]:
tokens = tokenizer.tokenize(sequence)

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
input_ids = torch.tensor([ids]).to(device)

In [ ]:
input_ids

In [ ]:
outputs = model(input_ids)

In [ ]:
outputs.logits

In [ ]:
inputs = tokenizer(sequence, return_tensors='pt').to(device)

In [ ]:
outputs = model(**inputs)

In [ ]:
outputs.logits

In [ ]:
model(torch.cat([input_ids, input_ids], dim=0)).logits

In [ ]:
tokenizer.pad_token_id, tokenizer.pad_token

In [ ]:
seq1 = [[200, 200, 200]]
seq2 = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

In [ ]:
for seq in [seq1, seq2, batched_ids]:
    print (model(torch.tensor(seq).to(device)).logits)

In [ ]:
attention_mask = [
    [1, 1, 1],
    [1, 1, 0]
]

In [ ]:
(model(torch.tensor(batched_ids).to(device), attention_mask=torch.tensor(attention_mask).to(device)).logits)

In [ ]:
tokenizer.model_max_length

### 2.6 Putting it all together

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint)

In [ ]:
model.to(device);

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

In [ ]:
# same as padding = True
inputs1 = tokenizer(sequences, padding='longest', return_tensors='pt').to(device)

In [ ]:
inputs1['input_ids'].shape

In [ ]:
inputs2 = tokenizer(sequences, padding='max_length', return_tensors='pt').to(device)

In [ ]:
inputs2['input_ids'].shape

In [ ]:
# pad sequences up to a specified max length
inputs3 = tokenizer(sequences, padding='max_length', max_length=8)

In [ ]:
# first sequence: the number of tokens is 16 so 0 padding are added to be 16
# second sequence: the number of tokens is 6 so 2 padding are added to be 8
len(inputs3['input_ids'][0]), len(inputs3['input_ids'][1])

In [ ]:
inputs4 = tokenizer(sequences, truncation=True)

In [ ]:
len(inputs4['input_ids'][0]), len(inputs4['input_ids'][1])

In [ ]:
inputs5 = tokenizer(sequences, truncation=True, max_length=8)

In [ ]:
# first sequence: the number of tokens is 16 and the max is 8 so the everything after 7 was truncated and an end token was added at the end
# second sequence: the number of tokens is 6 and the max is 8 so nothing was truncated
len(inputs5['input_ids'][0]), len(inputs5['input_ids'][1])

In [ ]:
inputs6 = tokenizer(sequences, padding=True, truncation=True, return_tensors='pt')

In [ ]:
inputs7 = tokenizer(sequences, padding=True, truncation=True, return_tensors='tf')

In [ ]:
inputs8 = tokenizer(sequences, padding=True, truncation=True, return_tensors='np')

In [ ]:
inputs6['input_ids'].dtype, inputs7['input_ids'].dtype, inputs8['input_ids'].dtype

In [ ]:
sequence = sequences[0]

In [ ]:
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
# token 101 [CLS] and 102 [SEP] are used to begin and end a sequence
# not all tokenizers have special tokens
inputs8['input_ids'][0], np.array(ids)

In [ ]:
tokenizer.decode(inputs8['input_ids'][0]), tokenizer.decode(ids)